In [1]:
import requests
import os 
import logging
import json 
import sys 

from pprint import pprint
from pathlib import Path
from glob import iglob, glob
from copy import deepcopy
from datetime import datetime

In [2]:
# this allows you to make changes and save in student_utils.py and the file is reloaded every time you run a code block
%load_ext autoreload    
%autoreload

# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Helper Functions 

In [3]:
def make_request(request_url: str, request_type: str, data: dict = None, **kwargs) -> requests.Response:

    verbose_text_fl: bool = kwargs.get('verbose_text_fl', True)

    if request_type == 'GET':
        response = requests.get(url=request_url)
        
    elif request_type == 'POST':
        response = requests.post(url=request_url, json=data)

    elif request_type == 'PUT':
        response = requests.put(url=request_url, json=data)

    else:
        raise NotImplementedError(f"Unknown value for parameter request_type: `{request_type}`.")
    
    print(f"Status code: {response.status_code}\n")

    if verbose_text_fl:
        print(f"Data: {response.text}\n")
    
    return response

def load_data(file_path: Path) -> dict:

    # READ file 
    with open(file_path, 'r') as f:
        fhir_data: dict = json.load(f)
    
    return fhir_data


# Basic requests

In [4]:
# DATA_DIR: str = Path('./data/')
DATA_DIR = Path(os.path.abspath("__file__"+"/../data"))

FHIR_BASE_URL: str = "http://localhost:8080/fhir"
# FHIR_BASE_URL: str = "http://host.docker.internal:8080/fhir"


In [5]:
r = requests.get(url=FHIR_BASE_URL+"/Bundle")
print(f"status code: {r.status_code}")

status code: 200


In [6]:
print(r.text)

{
  "resourceType": "Bundle",
  "id": "cbdbbbe4-9d0b-4cb7-8d2c-53128c0c6df0",
  "meta": {
    "lastUpdated": "2023-03-01T10:52:24.153+00:00"
  },
  "type": "searchset",
  "total": 0,
  "link": [ {
    "relation": "self",
    "url": "http://localhost:8080/fhir/Bundle"
  } ]
}


## Import resources
### Patient

Note: if we want to import resource directly with our custom ID (primary key) we must use `PUT` operation with the url form `<fhir_base>/Resource/[Resource ID]`. To see [details](https://github.com/hapifhir/hapi-fhir/issues/333)

More on [resource IDs](https://smilecdr.com/docs/fhir_repository/resource_ids.html)

In [7]:
req_url_patient: str = FHIR_BASE_URL + "/Patient/041520-1"
req_patient_data_path: Path = Path(DATA_DIR, 'resources/Patient-example.json')
req_patient_data: dict = load_data(req_patient_data_path)

_ = make_request(req_url_patient, 'PUT', req_patient_data)

Status code: 201

Data: {
  "resourceType": "Patient",
  "id": "041520-1",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:56:13.596+00:00"
  },
  "active": true,
  "name": [ {
    "family": "myProstate.eu"
  } ],
  "birthDate": "1955-01-01",
  "deceasedDateTime": "2019-12-11"
}



In [8]:
_ = make_request(req_url_patient, 'GET')

Status code: 200

Data: {
  "resourceType": "Patient",
  "id": "041520-1",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:56:13.596+00:00",
    "source": "#zRLGnds6tcN5RCFe"
  },
  "active": true,
  "name": [ {
    "family": "myProstate.eu"
  } ],
  "birthDate": "1955-01-01",
  "deceasedDateTime": "2019-12-11"
}



### Encounter

In [9]:
req_url_encounter: str = FHIR_BASE_URL + "/Encounter/A75E8847215B9A7525F586887806BA6B"
req_encounter_data_path: Path = Path(DATA_DIR, 'resources/Encounter-example.json')
req_encounter_data: dict = load_data(req_encounter_data_path)

_ = make_request(req_url_encounter, 'PUT', req_encounter_data)

Status code: 201

Data: {
  "resourceType": "Encounter",
  "id": "A75E8847215B9A7525F586887806BA6B",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:58:12.235+00:00"
  },
  "status": "finished",
  "subject": {
    "reference": "Patient/041520-1"
  },
  "period": {
    "start": "2015-01-15",
    "end": "2015-01-15"
  }
}



In [10]:
_ = make_request(req_url_encounter, 'GET')

Status code: 200

Data: {
  "resourceType": "Encounter",
  "id": "A75E8847215B9A7525F586887806BA6B",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:58:12.235+00:00",
    "source": "#fDq5TQfRMRF2ClhP"
  },
  "status": "finished",
  "subject": {
    "reference": "Patient/041520-1"
  },
  "period": {
    "start": "2015-01-15",
    "end": "2015-01-15"
  }
}



### Observation

In [11]:
req_url_observation: str = FHIR_BASE_URL + "/Observation/Observation00000000000000000006"
req_observation_data_path: Path = Path(DATA_DIR, 'resources/Observation-example.json')
req_observation_data: dict = load_data(req_observation_data_path)

_ = make_request(req_url_observation, 'PUT', req_observation_data)

Status code: 201

Data: {
  "resourceType": "Observation",
  "id": "Observation00000000000000000006",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:58:17.998+00:00"
  },
  "status": "final",
  "code": {
    "coding": [ {
      "system": "http://snomed.info/cst",
      "code": "102687007",
      "display": "Prostate specific antigen"
    }, {
      "system": "http://loinc.org",
      "code": "2857-1",
      "display": "Prostate specific Ag [Mass/volume] in Serum or Plasma"
    } ]
  },
  "subject": {
    "reference": "Patient/041520-1"
  },
  "effectiveDateTime": "2014-11-11",
  "valueQuantity": {
    "value": 5.77,
    "system": "http://unitsofmeasure.org",
    "code": "ng/mL"
  }
}



In [12]:
_ = make_request(req_url_observation, 'GET')

Status code: 200

Data: {
  "resourceType": "Observation",
  "id": "Observation00000000000000000006",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:58:17.998+00:00",
    "source": "#4iY4tXEL5fOURVqz"
  },
  "status": "final",
  "code": {
    "coding": [ {
      "system": "http://snomed.info/cst",
      "code": "102687007",
      "display": "Prostate specific antigen"
    }, {
      "system": "http://loinc.org",
      "code": "2857-1",
      "display": "Prostate specific Ag [Mass/volume] in Serum or Plasma"
    } ]
  },
  "subject": {
    "reference": "Patient/041520-1"
  },
  "effectiveDateTime": "2014-11-11",
  "valueQuantity": {
    "value": 5.77,
    "system": "http://unitsofmeasure.org",
    "code": "ng/mL"
  }
}



### Procedure

In [13]:
req_url_procedure: str = FHIR_BASE_URL + "/Procedure/Procedure000000000000000000027"
req_procedure_data_path: Path = Path(DATA_DIR, 'resources/Procedure-example.json')
req_procedure_data: dict = load_data(req_procedure_data_path)

_ = make_request(req_url_procedure, 'PUT', req_procedure_data)

Status code: 201

Data: {
  "resourceType": "Procedure",
  "id": "Procedure000000000000000000027",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:58:21.088+00:00"
  },
  "status": "completed",
  "code": {
    "coding": [ {
      "system": "http://snomed.info/cst",
      "code": "26294005",
      "display": "Radical prostatectomy (procedure)"
    } ]
  },
  "subject": {
    "reference": "Patient/041520-1"
  },
  "encounter": {
    "reference": "Encounter/A75E8847215B9A7525F586887806BA6B"
  },
  "performedPeriod": {
    "start": "2015-01-15",
    "end": "2015-01-15"
  }
}



In [14]:
_ = make_request(req_url_procedure, 'GET')

Status code: 200

Data: {
  "resourceType": "Procedure",
  "id": "Procedure000000000000000000027",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T10:58:21.088+00:00",
    "source": "#lmhoFrcBeHpB2El6"
  },
  "status": "completed",
  "code": {
    "coding": [ {
      "system": "http://snomed.info/cst",
      "code": "26294005",
      "display": "Radical prostatectomy (procedure)"
    } ]
  },
  "subject": {
    "reference": "Patient/041520-1"
  },
  "encounter": {
    "reference": "Encounter/A75E8847215B9A7525F586887806BA6B"
  },
  "performedPeriod": {
    "start": "2015-01-15",
    "end": "2015-01-15"
  }
}



### Bundle

In [15]:
req_url_bundle: str = deepcopy(FHIR_BASE_URL)
req_bundle_data_path: Path = Path(DATA_DIR, 'test-patients/toHapi/041520-12_fhir_v2.json')
req_bundle_data: dict = load_data(req_bundle_data_path)

_ = make_request(req_url_bundle, 'POST', req_bundle_data)

Status code: 200

Data: {
  "resourceType": "Bundle",
  "id": "22d7db94-3400-4930-8889-e58b2d8e509b",
  "type": "transaction-response",
  "link": [ {
    "relation": "self",
    "url": "http://localhost:8080/fhir"
  } ],
  "entry": [ {
    "response": {
      "status": "201 Created",
      "location": "Patient/041520-12/_history/1",
      "etag": "1",
      "lastModified": "2023-03-01T11:00:24.599+00:00",
      "outcome": {
        "resourceType": "OperationOutcome",
        "issue": [ {
          "severity": "information",
          "code": "informational",
          "details": {
            "coding": [ {
              "system": "https://hapifhir.io/fhir/CodeSystem/hapi-fhir-storage-response-code",
              "code": "SUCCESSFUL_UPDATE_AS_CREATE",
              "display": "Update as create succeeded."
            } ]
          },
          "diagnostics": "Successfully created resource \"Patient/041520-12/_history/1\" using update as create (ie. create with client assigned ID). Took

In [16]:
_ = make_request(FHIR_BASE_URL+'/Bundle', 'GET')

Status code: 200

Data: {
  "resourceType": "Bundle",
  "id": "a8c45104-fa04-41c7-8470-9e047b3235e9",
  "meta": {
    "lastUpdated": "2023-03-01T11:00:33.269+00:00"
  },
  "type": "searchset",
  "total": 0,
  "link": [ {
    "relation": "self",
    "url": "http://localhost:8080/fhir/Bundle"
  } ]
}



> Note: if we want to get the whole FHIR bundle of a patient we need to use keyword `$everything` 

> Note 2: we must pay attention about the pagination of the HAPI server. Usually, only some part of the data may be returned (e.g. 50). This can be changed in the HAPI settings 

In [17]:
req_url_patientEverything: str = FHIR_BASE_URL+'/Patient/041520-12/$everything'

bundle_04152012_response = make_request(req_url_patientEverything, 'GET')

Status code: 200

Data: {
  "resourceType": "Bundle",
  "id": "2c407383-cd21-4b1d-819e-8f0c58ad646c",
  "meta": {
    "lastUpdated": "2023-03-01T11:01:11.996+00:00"
  },
  "type": "searchset",
  "total": 16,
  "link": [ {
    "relation": "self",
    "url": "http://localhost:8080/fhir/Patient/041520-12/$everything"
  } ],
  "entry": [ {
    "fullUrl": "http://localhost:8080/fhir/Patient/041520-12",
    "resource": {
      "resourceType": "Patient",
      "id": "041520-12",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2023-03-01T11:00:24.599+00:00",
        "source": "#BHL1bO9QA5zcduTz"
      },
      "active": true,
      "name": [ {
        "family": "myProstate.eu"
      } ],
      "gender": "male",
      "birthDate": "1959-01-01"
    },
    "search": {
      "mode": "match"
    }
  }, {
    "fullUrl": "http://localhost:8080/fhir/Procedure/041520-711",
    "resource": {
      "resourceType": "Procedure",
      "id": "041520-711",
      "meta": {
        "versionId

In [64]:
bundle_04152012 = bundle_04152012_response.json()
print(f"type(bundle_04152012) : {type(bundle_04152012)}")
print(f"keys: {bundle_04152012.keys()}\n\n")
print(json.dumps(bundle_04152012.get('entry')[0], indent=4))

type(bundle_04152012) : <class 'dict'>
keys: dict_keys(['resourceType', 'id', 'meta', 'type', 'total', 'link', 'entry'])


{
    "fullUrl": "http://localhost:8080/fhir/Patient/041520-12",
    "resource": {
        "resourceType": "Patient",
        "id": "041520-12",
        "meta": {
            "versionId": "1",
            "lastUpdated": "2023-02-28T20:13:08.705+00:00",
            "source": "#QvDiJlIwm7pm6kcY"
        },
        "active": true,
        "name": [
            {
                "family": "myProstate.eu"
            }
        ],
        "gender": "male",
        "birthDate": "1959-01-01"
    },
    "search": {
        "mode": "match"
    }
}


## Insert FHIR Bundles

In [6]:
# DIR_PATH = os.path.dirname(os.path.realpath(__file__))
BUNDLE_DIR: Path = Path(DATA_DIR,'test-patients/toHapi')

bundles_paths: list = list(iglob(os.path.join(BUNDLE_DIR, '**/*.json'), recursive=True))

In [12]:
for i, single_path in enumerate(bundles_paths):

    if i > 1e5: 
        break

    start_processing = datetime.now()

    # Extract patient name 
    patient_id: str = single_path.split('/')[-1].split('.')[0]

    print(f"{i} - Processing bundle of patient : {patient_id}")
    # logger.info(f"{i} - Processing bundle of patient : {patient_id}")

    # READ file 
    fhir_bundle: dict = load_data(file_path=single_path)

    # Make request 
    response = make_request(request_url=FHIR_BASE_URL, 
                            request_type='POST', 
                            data=fhir_bundle, 
                            verbose_text_fl=False)

    # print(f"status code: {response.status_code}")
    # print(f"status code: {response.text}")

    print(f"{i} - Processing ends. Time - {datetime.now() - start_processing} s")

    
    # logger.info(f"{i} - Processing ends. Time - {datetime.now() - start_processing} s")

0 - Processing bundle of patient : 041520-12_fhir_v2
Status code: 200

status code: 200
0 - Processing ends. Time - 0:00:00.237395 s
1 - Processing bundle of patient : 041520-13_fhir_v2
Status code: 200

status code: 200
1 - Processing ends. Time - 0:00:00.184341 s
2 - Processing bundle of patient : 041520-1_fhir_v2
Status code: 200

status code: 200
2 - Processing ends. Time - 0:00:06.519203 s
3 - Processing bundle of patient : 041520-37_fhir_v2
Status code: 200

status code: 200
3 - Processing ends. Time - 0:00:08.114196 s
4 - Processing bundle of patient : 041520-38_fhir_v2
Status code: 200

status code: 200
4 - Processing ends. Time - 0:00:05.938473 s
5 - Processing bundle of patient : 041520-39_fhir_v2
Status code: 200

status code: 200
5 - Processing ends. Time - 0:00:02.834275 s
6 - Processing bundle of patient : 041520-43_fhir_v2
Status code: 200

status code: 200
6 - Processing ends. Time - 0:00:06.089532 s
7 - Processing bundle of patient : 041520-45_fhir_v2
Status code: 200


The FHIR resources are already created 

* URI - ../_history/1
* response.status: 200 OK (instead of 201 Created)

# Structure Definitions

In [18]:
req_url_sd_patient: str = FHIR_BASE_URL + "/StructureDefinition/fhir-workshop-patient"
req_sd_patient_data_path: Path = Path(DATA_DIR, 'fsh-generated/resources/StructureDefinition-shs-codecon-patient.json')
req_sd_patient_data: dict = load_data(req_sd_patient_data_path)

_ = make_request(req_url_sd_patient, 'PUT', req_sd_patient_data)

Status code: 201

Data: {
  "resourceType": "StructureDefinition",
  "id": "fhir-workshop-patient",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T11:02:10.758+00:00"
  },
  "url": "http://fhir-workshop.org/StructureDefinition/fhir-workshop-patient",
  "version": "0.1.0",
  "name": "FHIRWorkshop_CLINIC51_Patient",
  "title": "FHIRWorkshop - CLINIC 5.1 -  Patient",
  "status": "draft",
  "description": "Basic descriptive information to identify the patient, like patient name, patient identifier, birthdate / age, sex.",
  "fhirVersion": "4.0.1",
  "mapping": [ {
    "identity": "rim",
    "uri": "http://hl7.org/v3",
    "name": "RIM Mapping"
  }, {
    "identity": "cda",
    "uri": "http://hl7.org/v3/cda",
    "name": "CDA (R2)"
  }, {
    "identity": "w5",
    "uri": "http://hl7.org/fhir/fivews",
    "name": "FiveWs Pattern Mapping"
  }, {
    "identity": "v2",
    "uri": "http://hl7.org/v2",
    "name": "HL7 v2 Mapping"
  }, {
    "identity": "loinc",
    "uri": "http

In [19]:
_ = make_request(req_url_sd_patient, 'GET')

Status code: 200

Data: {
  "resourceType": "StructureDefinition",
  "id": "fhir-workshop-patient",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2023-03-01T11:02:10.758+00:00",
    "source": "#Wi2HJcJ9M0ouPmvt"
  },
  "url": "http://fhir-workshop.org/StructureDefinition/fhir-workshop-patient",
  "version": "0.1.0",
  "name": "FHIRWorkshop_CLINIC51_Patient",
  "title": "FHIRWorkshop - CLINIC 5.1 -  Patient",
  "status": "draft",
  "description": "Basic descriptive information to identify the patient, like patient name, patient identifier, birthdate / age, sex.",
  "fhirVersion": "4.0.1",
  "mapping": [ {
    "identity": "rim",
    "uri": "http://hl7.org/v3",
    "name": "RIM Mapping"
  }, {
    "identity": "cda",
    "uri": "http://hl7.org/v3/cda",
    "name": "CDA (R2)"
  }, {
    "identity": "w5",
    "uri": "http://hl7.org/fhir/fivews",
    "name": "FiveWs Pattern Mapping"
  }, {
    "identity": "v2",
    "uri": "http://hl7.org/v2",
    "name": "HL7 v2 Mapping"
  }, {
    "i

## fhirclient package

In [20]:
from fhirclient import client
from fhirclient import server
from fhirclient.models.patient import Patient
from fhirclient.models import bundle

In [21]:
fhirclient_settings: dict = {
    'app_id': 'my_web_app',
    'api_base': FHIR_BASE_URL
}

smart = client.FHIRClient(settings=fhirclient_settings)
server_api = server.FHIRServer(client=smart, base_uri=FHIR_BASE_URL)



### Patient

In [22]:
query: str = f'Patient'
res = server_api.request_json(query)

# print(res)
print(type(res))
print(res.keys())
print(type(res.get('entry')))

<class 'dict'>
dict_keys(['resourceType', 'id', 'meta', 'type', 'total', 'link', 'entry'])
<class 'list'>


### Observation

In [12]:
#%% Observation 
query: str = f'Observation?code=102687007'

res = server_api.request_json(query)
print(type(res))
print(json.dumps(res, indent=4))

<class 'dict'>
{
    "resourceType": "Bundle",
    "id": "1331d499-b327-4afd-a583-ce07927f97c4",
    "meta": {
        "lastUpdated": "2023-03-01T08:24:02.042+00:00"
    },
    "type": "searchset",
    "link": [
        {
            "relation": "self",
            "url": "http://localhost:8080/fhir/Observation?code=102687007"
        },
        {
            "relation": "next",
            "url": "http://localhost:8080/fhir?_getpages=1331d499-b327-4afd-a583-ce07927f97c4&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
        }
    ],
    "entry": [
        {
            "fullUrl": "http://localhost:8080/fhir/Observation/Observation00000000000000000006",
            "resource": {
                "resourceType": "Observation",
                "id": "Observation00000000000000000006",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2023-02-28T20:06:34.171+00:00",
                    "source": "#T8eRKN8bicZiqjPn"
        